In [26]:
import torch 
import torch.nn as nn
import torch.functional as F 
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt 

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print(f"Device : {device}")

Device : cuda


In [28]:
#Hyperparameters

num_epochs = 4
image_size = [32,32]
input_size = [3,32,32]
input_channels = input_size[0]
num_of_classes = 10
learning_rate = 0.001
batch_size = 8

In [29]:
#Transforms

data_transforms = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=data_transforms)
test_dataset = torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=data_transforms)

train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader =  DataLoader(test_dataset,batch_size=batch_size,shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [30]:
dataiter = iter(train_loader)
data_batch = dataiter.next()

inputs,labels = data_batch
print(inputs.shape)
print(labels.shape)



torch.Size([8, 3, 32, 32])
torch.Size([8])


In [31]:
classes = ('plane','car','bird',"cat","deer","dog","frog","horse","ship","truck")

In [47]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        self.conv2d1 = nn.Conv2d(in_channels=3,out_channels=6,kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2,stride=2)
        self.relu = nn.ReLU()
        self.conv2d2 = nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5)
        self.fc1 = nn.Linear(in_features=16*5*5,out_features=120)
        self.fc2 = nn.Linear(in_features=120,out_features=64)
        self.fc3 = nn.Linear(in_features=64,out_features=num_of_classes)

    def forward(self,inputs):
        out = self.conv2d1(inputs) #convolution
        out = self.relu(out) #activaltion
        out = self.pool(out) #pooling
        out = self.conv2d2(out) #convolution
        out = self.relu(out) #activaltion
        out = self.pool(out) #pooling
        out = torch.flatten(out,start_dim=1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        # out = nn.Softmax(out) # we do not do this as we are using cross entropy loss function, which does softmax
        return out

In [48]:
model = ConvNet().to(device)

loss_func = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [50]:
steps_per_epoch = len(train_loader)

for epoch in range(num_epochs):
    for i,(inputs,labels) in enumerate(train_loader):
        #print(inputs.shape) # 8 3 32 32
        #print(labels.shape) # 8

        inputs = inputs.to(device)
        labels = labels.to(device)

        #forward pass
        y_pred = model(inputs)
        loss = loss_func(y_pred,labels)

        #backward pass
        optimiser.zero_grad()
        loss.backward()

        #weight updation
        optimiser.step()

        if (i+1)%1000 == 0:
            print(f"Epoch {epoch+1} / {num_epochs}, Step {i+1}/{steps_per_epoch}, Loss is {loss}")


Epoch 1 / 4, Step 1000/6250, Loss is 2.293445348739624
Epoch 1 / 4, Step 2000/6250, Loss is 2.287079334259033
Epoch 1 / 4, Step 3000/6250, Loss is 2.2919352054595947
Epoch 1 / 4, Step 4000/6250, Loss is 2.292660713195801
Epoch 1 / 4, Step 5000/6250, Loss is 2.2993009090423584
Epoch 1 / 4, Step 6000/6250, Loss is 2.292539119720459
Epoch 2 / 4, Step 1000/6250, Loss is 2.2324464321136475
Epoch 2 / 4, Step 2000/6250, Loss is 2.2086026668548584
Epoch 2 / 4, Step 3000/6250, Loss is 2.087015390396118
Epoch 2 / 4, Step 4000/6250, Loss is 2.3412258625030518
Epoch 2 / 4, Step 5000/6250, Loss is 2.1050918102264404
Epoch 2 / 4, Step 6000/6250, Loss is 2.1587648391723633
Epoch 3 / 4, Step 1000/6250, Loss is 2.439683437347412
Epoch 3 / 4, Step 2000/6250, Loss is 2.4590976238250732
Epoch 3 / 4, Step 3000/6250, Loss is 2.245692729949951
Epoch 3 / 4, Step 4000/6250, Loss is 1.9949367046356201
Epoch 3 / 4, Step 5000/6250, Loss is 2.229139566421509
Epoch 3 / 4, Step 6000/6250, Loss is 1.8787338733673096


In [51]:
# Replace this section with compact skearn implementation, confusion matrices

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 34.61 %
Accuracy of plane: 36.2 %
Accuracy of car: 47.0 %
Accuracy of bird: 8.7 %
Accuracy of cat: 2.9 %
Accuracy of deer: 12.3 %
Accuracy of dog: 45.8 %
Accuracy of frog: 59.1 %
Accuracy of horse: 39.8 %
Accuracy of ship: 55.0 %
Accuracy of truck: 39.3 %
